## Note

This code was used for exploratory purposes early on in our analysis. The intention was to see if modeling each expert as a gaussian process could provide predictions for new nocs. We decided to not go down these route due to the severly decreased accuracy compared to our main gaussian process model. However, that does not mean that this approach will not work. We just did not prioritize trying to make it work. As such, this code is provided as-is. It currently does not work and will probably require some significant changes to make it work. Additionally, when running the models, we could only run one model before having to restart our kernel in order to prevent the kernel from dying for the second model. Also note that this code uses multiprocessing which can make debugging difficult.

## Setup

In [4]:
%run ../utils/gaussian_utils.ipynb

from pathos.multiprocessing import ProcessingPool as Pool

In [2]:
def test_with_experts(x_all, y_all, kern, name, agg_level, dist):
    params = '_'.join([name, 'gp_experts'])
    print('Testing:', params)
    start = time.time()
    x, y = x_and_y(x_all, y_all)
    result = cv_experts(x, y, x_all, y_all, kern, agg_level, dist)
    end = time.time()
    print('Accuracy:', result['Avg. score'])
    print('MSE:', result['Avg. mse'])
    print('ROC:', result['Avg. roc'])
    print('Confusion:', result['Confusion'])
    print('Time elapsed (sec):', end - start)
    pickle_results(result, params)
    print('Results saved to disk')

In [3]:
# do cross validation
def cv_experts(x, y, x_all, y_all, kern, agg_level, dist):
    x_all = np.asarray(x_all.drop(['noc', 'participant.id'], axis=1))
    group = x_cont_ind['noc']
    kf = GroupKFold(n_splits=5)
    scores = []
    results = []
    mse = []
    roc = []
    total_confusion = defaultdict(lambda: defaultdict(int))

    for train_index, test_index in kf.split(x, groups=group):
        start = time.time()
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        x_test_all = x_all[test_index]
        densities, unique_indices = gp_experts(x_train, y_train, y_all, kern, x_test)
        scores.append(accuracy([y_test[i] for i in unique_indices], densities))
        mse_error, roc_error, confusion = calc_errors([x_test_all[i] for i in unique_indices], 
                                                      densities, 
                                                      agg_level, 
                                                      dist)
        mse.append(mse_error)
        roc.append(roc_error)
        for y_t in confusion:
            for y_p in confusion[y_t]:
                total_confusion[y_t][y_p] += confusion[y_t][y_p]
        results.append(create_results([x_test[i] for i in unique_indices], densities))
        end = time.time()
        print('Fold tested in (sec):', end - start)

    return {'Avg. score':  sum(scores)/len(scores),
            'Avg. mse':    sum(mse)/len(mse),
            'Avg. roc':    sum(roc)/len(roc),
            'Confusion':   dict(total_confusion),
            'All scores':  scores,
            'All mse':     mse,
            'All roc':     roc,
            'All results': results}

In [4]:
def gp_experts(x_train, y_train, y_all, kern, x_test):
    def tester(model):
        densities = test_model(model, [x_test[i] for i in unique_indices])
        preds = [np.argmax(d) for d in densities]
        return preds
    
    def builder(params):
        expert, data = params
        kernel = get_kern(kern)
        likelihood = create_likelihood(y_all)
        model = build_model(np.asarray(data),
                            ys[workshop][expert],
                            likelihood,
                            kernel)
        preds = tester(model)
        classes = np.max(model.Y.value + 1)
        del model
        return workshop, expert, classes, preds
    # Done to drop the first 3 rows of the data (workshop num, noc name and participant id)
    # Set up test data
    x_test = np.asarray([np.asarray(x[3:]).astype(float) for x in x_test])
    unique, unique_indices = np.unique(x_test, axis=0, return_index=True)
    # Set up training data
    xs = defaultdict(lambda: defaultdict(list))
    ys = defaultdict(lambda: defaultdict(list))
    for index, x in enumerate(x_train):
        xs[x[0]][x[2]].append(np.asarray(x[3:]).astype(float))
        ys[x[0]][x[2]].append(y_train[index])
    for workshop in xs:
        with Pool() as pool:
            results = pool.map(builder, xs[workshop].items())
    experts = 0
    densities = []
    for i in range(len(results[0][3])):
        density = []
        for j in range(int(results[0][2])):
            density.append([0])
        densities.append(density)
    for workshop, expert, classes, preds in results:
        experts += 1
        for index, pred in enumerate(preds):
            densities[index][pred][0] += 1
    for row in densities:
        for d in row:
            d[0] = d[0] / experts
    return densities, unique_indices

## Run through - experts

## Scratchpad